In [2]:
#!/usr/bin/python3
import requests
import logging
import os
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from urllib.parse import urlparse
import re

logging.basicConfig(filename="log.txt",
                    filemode='a',
                    format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.DEBUG)
logging.info("Starting download session")

# GET DATASET URLS
req = Request("https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page")
html = urlopen(req)

soup = BeautifulSoup(html, "lxml")
all_urls = []
for link in soup.findAll('a'):
    all_urls.append(link.get('href'))

# all the data is hosted on cloudfront urls only
cloudfront_urls = [link for link in all_urls if "cloudfront" in link]

# DOWNLOAD DATASET
def download_file_from_url(url):
    filename = os.path.basename(urlparse(url).path)
    logging.info(f"Started {filename} from {url}")
    resp = requests.get(url)
    with open(filename, "wb") as f:
        f.write(resp.content)
    logging.info(f"Finished {filename} from {url}")

# download files SEQUENTIALLY
for url in cloudfront_urls:
    download_file_from_url(url=url)